In [1]:
import numpy as np
from tempfile import gettempdir
import biotite.structure.io.pdb as pdb
import biotite.database.rcsb as rcsb

In [2]:
def centroid(coords):
    """Compute the geometric center (centroid) of a set of coordinates."""
    return np.mean(coords, axis=0)

def kabsch_rotation_matrix(A, B):
    """Return a rotation matrix using the Kabsch algorithm."""
    # Compute the covariance matrix
    C = np.dot(np.transpose(A), B)
    
    # Singular value decomposition
    V, S, Wt = np.linalg.svd(C)
    
    # Ensure a right-handed coordinate system
    d = (np.linalg.det(V) * np.linalg.det(Wt)) < 0.0
    
    if d:
        S[-1] = -S[-1]
        V[:,-1] = -V[:,-1]
    
    # Compute the rotation matrix
    U = np.dot(V, Wt)
    return U

def rotation_matrix_to_euler_angles(R):
    """
    Convert a rotation matrix to Euler angles (XYZ order).
    
    Parameters:
    R (ndarray): A 3x3 rotation matrix.

    Returns:
    tuple: A tuple of three Euler angles (alpha, beta, gamma).
    """
    assert R.shape == (3, 3)
    
    beta = np.arctan2(-R[2, 0], np.sqrt(R[2, 1]**2 + R[2, 2]**2))
    alpha = np.arctan2(R[2, 1], R[2, 2])
    gamma = np.arctan2(R[1, 0], R[0, 0])
    
    return alpha, beta, gamma

In [3]:
pdb_file_path = rcsb.fetch("1ubi", "pdb", gettempdir())
target_pdb_file = pdb.PDBFile.read(pdb_file_path)
target_structure = target_pdb_file.get_structure()

# get the alpha carbon coordinates
target_CA_idx = np.argwhere(target_structure.atom_name=="CA")
target_CA_idx = target_CA_idx.reshape((-1,))
print(np.shape(target_CA_idx))

(76,)


In [4]:
pdb_file_path = rcsb.fetch("6tuv", "pdb", gettempdir())
mobile_pdb_file = pdb.PDBFile.read(pdb_file_path)
mobile_structure = mobile_pdb_file.get_structure()

# get the alpha carbon coordinates
mobile_CA_idx = np.argwhere(mobile_structure.atom_name=="CA")
mobile_CA_idx = mobile_CA_idx.reshape((-1,))

# can also add more filters like selecting chain D
temp_idx = np.argwhere(mobile_structure.chain_id[mobile_CA_idx]=="D")
temp_idx = temp_idx.reshape((-1,))
mobile_CA_idx = mobile_CA_idx[temp_idx]
print(np.shape(mobile_CA_idx))

(76,)


In [5]:
mobile_atom_coords = mobile_structure.coord[0,mobile_CA_idx,:]
target_atom_coords = target_structure.coord[0,target_CA_idx,:]

# Translate both A and B to their centroids
target_centroid = centroid(target_atom_coords)
mobile_centroid = centroid(mobile_atom_coords)
A2 = target_atom_coords.copy()
B2 = mobile_atom_coords.copy()

print(f"Translate target molecule by x: {-target_centroid[0]/100} y: {-target_centroid[1]/100} z: {-target_centroid[2]/100} to align to center")
print(f"Translate mobile molecule by x: {-mobile_centroid[0]/100} y: {-mobile_centroid[1]/100} z: {-mobile_centroid[2]/100} to align to center")

Translate target molecule by x: -0.3044176292419434 y: -0.29000947952270506 z: -0.1552194881439209 to align to center
Translate mobile molecule by x: -0.25101184844970703 y: -0.21613740921020508 z: -0.3967397689819336 to align to center


In [6]:
# move the protein's center of mass to 0,0,0
A2 -= target_centroid
B2 -= mobile_centroid

# Compute the optimal rotation matrix
U = kabsch_rotation_matrix(A2, B2)

# Get Euler angles
alpha, beta, gamma = rotation_matrix_to_euler_angles(U)
# Convert to degrees if needed
alpha_deg = np.degrees(alpha)
beta_deg = np.degrees(beta)
gamma_deg = np.degrees(gamma)

print(f"Rotate mobile molecule about X (alpha): {alpha_deg:.2f}°")
print(f"Rotate mobile molecule about Y (beta): {beta_deg:.2f}°")
print(f"Rotate mobile molecule about Z (gamma): {gamma_deg:.2f}°")

Rotate mobile molecule about X (alpha): -96.15°
Rotate mobile molecule about Y (beta): -32.74°
Rotate mobile molecule about Z (gamma): 154.18°


If you don't want to translate your target molecule, then you can translate your mobile molecule (after you rotate your mobile molecule) by the opposite of what you would have translated your target molecule (shown above "Translate target molecule by...")

In [9]:
# translate your mobile molecule to your target molecule: 
print(f"OPTIONAL. If you don't want your molecules to be centered then: ")
print(f"Translate mobile molecule by x: {target_centroid[0]/100} y: {target_centroid[1]/100} z: {target_centroid[2]/100} to align with target molecule centroid")


OPTIONAL. If you don't want your molecules to be centered then: 
Translate mobile molecule by x: 0.3044176292419434 y: 0.29000947952270506 z: 0.1552194881439209 to align with target molecule centroid
